In [10]:
import networkx as nx
import numpy as np

In [9]:
G = nx.read_edgelist('graph.edgelist')

In [19]:
edges = np.random.choice(G.edges(), 100, replace=False)
H = G.edge_subgraph(edges)
nx.draw()

ValueError: a must be 1-dimensional